In [7]:
import time
from time import perf_counter

x= perf_counter()
time.sleep(1)
y=perf_counter()
t=y-x
print(f'sum = {sum}, time taken = {t}')


sum = <built-in function sum>, time taken = 1.0001898999980767


In [8]:
from dotenv import load_dotenv
load_dotenv()
from langchain_openai import OpenAIEmbeddings
# from langchain.llms import OpenAI
from langchain_openai import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.callbacks import get_openai_callback

In [9]:
from langchain import PromptTemplate
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import CTransformers
from langchain.chains import RetrievalQA

In [10]:
# model_name = "sentence-transformers/all-mpnet-base-v2"
# embeddings = HuggingFaceEmbeddings(model_name=model_name)

embeddings = OpenAIEmbeddings()

DB_FAISS_PATH = "vectorstores/db_faiss"
db= FAISS.load_local(DB_FAISS_PATH, embeddings)

In [11]:
# load llm 

# llm = CTransformers(model = "llama-2-7b-chat.ggmlv3.q8_0.bin",model_type = "llama",max_new_tokens= 512,temperature= 0.5)
llm = OpenAI()


In [12]:
custom_prompt_template = """
Use the following pieces of information to answer the user's question. 
If you do not know th answer, please just say that you do not know the answer. Do not try to make up the answer.
Context: {context}
Question: {question}
Only returns the helpful answer below and nothing else.
Helpful answer:
"""

qa_prompt = PromptTemplate(template=custom_prompt_template, input_variables=['context', 'question'])

# retrieval qa chain
qa_chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type = "stuff",
        retriever=db.as_retriever(search_kwargs={'k': 2}),
        return_source_documents=True,
        chain_type_kwargs={'prompt':qa_prompt}
    )

In [26]:
query = "What is diabetes?"

# docsearch = FAISS.from_texts(text_chunks, embeddings)
docsearch = FAISS.load_local(DB_FAISS_PATH, embeddings)

a=perf_counter()

#res = qa_chain.invoke(db, query)
new_db = FAISS.load_local("faiss_index", embeddings)
docs = new_db.similarity_search(query)

b=perf_counter()
t=b-a
print(f'time taken {t}')

time taken 1.3541780000014114


In [27]:
chain = load_qa_chain(llm, chain_type="stuff")
# res = chain.run(input_documents=docs, question=query)
with get_openai_callback() as cb:
    res = chain.run(input_documents=docs, question=query)
    print(cb)

Tokens Used: 320
	Prompt Tokens: 277
	Completion Tokens: 43
Successful Requests: 1
Total Cost (USD): $0.0005015000000000001


In [28]:
print(res)

 Diabetes is a disease that affects the body's ability to regulate sugar levels and produce insulin. It can result in damage to blood vessels and other complications. It is often diagnosed through blood tests that show high sugar levels.
